In [169]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import tqdm

from gensim.models import Word2Vec
from gensim.models.fasttext import FastText

from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import *

import xgboost as xgb

import warnings
warnings.filterwarnings("ignore")

num_i = 2

In [170]:
train_df = pd.read_csv('/Users/nitanshjain/Documents/Projects/CASE/codefiles/subtask_1_data/train_subtask1_preprocessed_{}.csv'.format(num_i))
print(train_df.head())

               index                                               text  label
0     train_01_0_892  the state alleged they hacked sabata petros ch...      1
1    train_01_1_2714  chale was allegedly chased group about thirty ...      0
2   train_01_10_2619  the farmworkers strike resumed tuesday when th...      1
3  train_01_100_2680  demonstrators have filed for permit hold rally...      1
4  train_01_101_3090  footage the attack which included pregnant wom...      1


In [171]:
dev_df = pd.read_csv('/Users/nitanshjain/Documents/Projects/CASE/codefiles/subtask_1_data/dev_subtask1_preprocessed_{}.csv'.format(num_i))
dev_df.head()

dev_df_labels = pd.read_csv('/Users/nitanshjain/Documents/Projects/CASE/tanfiona CausalNewsCorpus master data-V2/dev_subtask1.csv')
labels = dev_df_labels['label'].values
del(dev_df_labels)
print(labels)
print(dev_df.head())

[1 1 0 1 1 1 1 1 0 1 0 1 1 1 1 0 1 1 1 1 1 1 0 1 1 1 1 0 1 0 1 1 1 1 0 1 0
 0 1 1 1 0 1 1 0 0 1 0 0 0 1 1 0 1 1 0 1 0 1 1 0 0 0 1 0 0 0 0 1 1 1 1 0 0
 1 0 0 1 1 0 1 1 0 1 0 1 0 0 0 1 0 0 0 1 1 1 1 0 0 1 1 1 1 0 0 1 1 1 1 0 1
 0 1 0 1 1 0 1 0 1 1 1 0 1 0 0 1 1 1 0 0 0 1 1 1 1 1 1 0 1 1 1 0 0 0 1 0 1
 0 0 0 0 0 0 0 0 1 1 1 1 1 1 0 1 0 1 0 0 1 1 0 0 1 0 1 0 0 0 0 1 1 1 0 0 0
 0 1 1 1 0 1 1 0 1 0 0 1 1 0 0 0 0 0 0 0 1 1 1 1 1 1 1 0 1 1 1 1 0 1 0 1 0
 0 1 1 1 0 0 0 1 1 1 1 1 0 0 1 0 1 0 0 0 0 1 0 1 1 0 1 0 0 1 1 1 0 0 0 1 1
 0 1 1 0 0 1 0 0 0 1 1 0 0 1 1 1 0 1 0 1 0 1 1 1 0 0 0 0 1 0 1 0 0 0 1 0 0
 1 1 1 0 1 1 0 0 1 0 1 0 0 0 1 0 0 1 1 1 1 1 1 0 0 1 0 1 0 1 0 0 1 1 1 1 0
 0 1 0 1 1 1 0]
               index                                               text
0    train_10_0_2136  the movement was catapulted into the headlines...
1     train_10_1_350  several thousand protesters took the streets a...
2   train_10_10_3104  the protest not just about saving medha life b...
3  train_10_100_1188 

In [172]:
embedding_size = 100
window_size = 2
min_word = 2
down_sampling = 1e-2

fast_text_model_train = FastText(train_df.text.tolist(),
                      window=window_size,
                      min_count=min_word,
                      sample=down_sampling,
                      workers=4,
                      sg=1)

fast_text_model_dev = FastText(dev_df.text.tolist(),
                      window=window_size,
                      min_count=min_word,
                      sample=down_sampling,
                      workers=4,
                      sg=1)

fast_text_model_train.save("/Users/nitanshjain/Documents/Projects/CASE/codefiles/subtask_1_embeddings/ft_model_train_text_{}".format(num_i))
fast_text_model_train = Word2Vec.load("/Users/nitanshjain/Documents/Projects/CASE/codefiles/subtask_1_embeddings/ft_model_train_text_{}".format(num_i))

fast_text_model_dev.save("/Users/nitanshjain/Documents/Projects/CASE/codefiles/subtask_1_embeddings/ft_model_dev_text_{}".format(num_i))
fast_text_model_dev = Word2Vec.load("/Users/nitanshjain/Documents/Projects/CASE/codefiles/subtask_1_embeddings/ft_model_dev_text_{}".format(num_i))

In [ ]:
def create_file(create_file, model_file, x):
    fast_text_model = Word2Vec.load(model_file)
    
    with open(create_file, 'w+') as word2vec_file:
        for index, row in x.iterrows():
            model_vector = (np.mean([fast_text_model.wv[token] for token in row['text']], axis=0)).tolist()
            if index == 0:
                header = ",".join(str(ele) for ele in range(100))
                word2vec_file.write(header)
                word2vec_file.write("\n")
            
            if type(model_vector) is list:
                line1 = ",".join( [str(vector_element) for vector_element in model_vector] )
            else:
                line1 = ",".join([str(0) for i in range(100)])
            word2vec_file.write(line1)
            word2vec_file.write('\n')
    
    df = pd.read_csv(create_file)
    return df

fast_text_train_filename = '/Users/nitanshjain/Documents/Projects/CASE/codefiles/subtask_1_data/train_ft_{}.csv'.format(num_i)
fast_text_train_model_file = '/Users/nitanshjain/Documents/Projects/CASE/codefiles/subtask_1_embeddings/ft_model_train_text_{}'.format(num_i)
fast_text_train_embeddings_df = create_file(fast_text_train_filename, fast_text_train_model_file, train_df)

fast_text_train_embeddings = fast_text_train_embeddings_df.values

print(fast_text_train_embeddings_df.shape)
fast_text_train_embeddings_df.head()


fast_text_dev_filename = '/Users/nitanshjain/Documents/Projects/CASE/codefiles/subtask_1_data/dev_ft_{}.csv'.format(num_i)
fast_text_dev_model_file = '/Users/nitanshjain/Documents/Projects/CASE/codefiles/subtask_1_embeddings/ft_model_dev_text_{}'.format(num_i)
fast_text_dev_embeddings_df = create_file(fast_text_dev_filename, fast_text_dev_model_file, dev_df)

fast_text_dev_embeddings = fast_text_dev_embeddings_df.values

print(fast_text_dev_embeddings_df.shape)
fast_text_dev_embeddings_df.head()

(3075, 100)
(340, 100)


,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,-0.039237,-0.013505,0.012205,0.045884,0.003156,-0.039627,0.030083,0.112988,-0.087768,-0.034023,...,0.019219,0.013415,0.041315,-0.048878,0.107522,-0.006740,-0.033759,0.060725,-0.042516,0.004475
1,-0.036355,-0.008497,0.017878,0.053817,0.004796,-0.041806,0.031525,0.107489,-0.086755,-0.034586,...,0.014588,0.016184,0.031874,-0.042705,0.103415,-0.005732,-0.035012,0.064278,-0.046546,0.016654
2,-0.042594,-0.017009,0.008300,0.049535,0.020627,-0.041278,0.031622,0.104239,-0.084942,-0.030426,...,0.018115,0.019635,0.038585,-0.041442,0.099792,-0.018333,-0.029365,0.071848,-0.046345,0.005251
3,-0.044937,-0.010201,0.016283,0.050464,-0.003600,-0.041216,0.032073,0.110979,-0.087425,-0.037058,...,0.015697,0.014080,0.035600,-0.046887,0.106060,-0.004090,-0.040275,0.061485,-0.048366,0.015961
4,-0.039444,-0.013691,0.011223,0.046879,0.004982,-0.041318,0.029821,0.109936,-0.088513,-0.025932,...,0.022621,0.016028,0.039806,-0.045083,0.106328,-0.003006,-0.034226,0.059881,-0.047962,0.008989


In [ ]:
x = fast_text_train_embeddings_df.values
y = train_df['label'].values

mm = MinMaxScaler()
x = mm.fit_transform(x)

count_0 = np.unique(y, return_counts=True)[1][0]
count_1 = np.unique(y, return_counts=True)[1][1]
estimate = count_0/count_1

In [ ]:
xgb_model = xgb.XGBClassifier(scale_pos_weight=estimate)

parameters = {
            'objective':['binary:logistic'],
            'learning_rate': [0.1, 0.01, 0.001, 0.0001], 
            'max_depth': [6, 7, 8],
            'n_estimators': [1000], #number of trees, change it to 1000 for better results
            'seed': [1337]
        }

clf = GridSearchCV(xgb_model, parameters, n_jobs=5, 
                   cv=StratifiedKFold(n_splits=3, random_state=42, shuffle=True), 
                   verbose=0, refit=True)

clf.fit(x, y)
print(clf.best_params_, clf.best_score_)

x_dev = fast_text_dev_embeddings_df.values
y_dev = labels

y_pred = clf.predict(x_dev)
print(classification_report(y_dev, y_pred))
print(f1_score(y_dev, y_pred))
print(recall_score(y_dev, y_pred))
print(precision_score(y_dev, y_pred))

{'learning_rate': 0.01, 'max_depth': 6, 'n_estimators': 1000, 'objective': 'binary:logistic', 'seed': 1337} 0.6035772357723577
              precision    recall  f1-score   support

           0       0.46      1.00      0.63       155
           1       0.00      0.00      0.00       185

    accuracy                           0.46       340
   macro avg       0.23      0.50      0.31       340
weighted avg       0.21      0.46      0.29       340

0.0
0.0
0.0


In [161]:
mnb = MultinomialNB()

parameters = {
            'fit_prior': [True, False],
            'class_prior': [None, [0.5, 0.5], [0.6, 0.4], [0.4, 0.6]]
        }

mnb_gsc = GridSearchCV(mnb, parameters, n_jobs=5, 
                   cv=StratifiedKFold(n_splits=3, random_state=42, shuffle=True), 
                   verbose=0, refit=True)

mnb_gsc.fit(x, y)
print(mnb_gsc.best_params_, mnb_gsc.best_score_)

x_dev = fast_text_dev_embeddings_df.values
y_dev = labels

y_pred = mnb_gsc.predict(x_dev)
print(classification_report(y_dev, y_pred))
print(f1_score(y_dev, y_pred))
print(recall_score(y_dev, y_pred))
print(precision_score(y_dev, y_pred))

{'class_prior': None, 'fit_prior': False} 0.566178861788618
              precision    recall  f1-score   support

           0       0.45      0.99      0.62       155
           1       0.00      0.00      0.00       185

    accuracy                           0.45       340
   macro avg       0.23      0.50      0.31       340
weighted avg       0.21      0.45      0.28       340

0.0
0.0
0.0
